# Project Introduction: Integrating Stochastic Oscillator and Average True Range (ATR) for Stock Analysis

## Objective
The objective of this project is to develop a reliable stock screening tool that identifies potential buy and sell signals using two powerful technical indicators: the Stochastic Oscillator and the Average True Range (ATR). By combining these indicators, we aim to enhance the accuracy of our signals, leveraging the strengths of both momentum and volatility analysis.

## Background
Technical analysis involves using historical price and volume data to forecast future price movements. Among the myriad of technical indicators available, the Stochastic Oscillator and ATR are two widely respected tools that traders use to assess market conditions.

## Why Stochastic Oscillator and ATR?
### Stochastic Oscillator
- **Purpose**: The Stochastic Oscillator is used to measure the momentum of price movements and identify potential overbought or oversold conditions.
- **Components**:
  - **%K Line**: The main line, typically calculated over a 14-period interval.
  - **%D Line**: The signal line, which is a 3-period moving average of the %K line.
- **Signals**:
  - **Overbought Condition**: %K and %D lines above 80, indicating potential for a price reversal to the downside.
  - **Oversold Condition**: %K and %D lines below 20, indicating potential for a price reversal to the upside.
  - **Buy Signal**: %K line crosses above the %D line in the oversold region.
  - **Sell Signal**: %K line crosses below the %D line in the overbought region.

### Average True Range (ATR)
- **Purpose**: The ATR is used to measure market volatility by examining the range of price movements over a given period.
- **Components**:
  - **True Range (TR)**: The greatest of the following:
    - Current high minus the current low
    - Absolute value of the current high minus the previous close
    - Absolute value of the current low minus the previous close
  - **ATR Line**: A moving average of the True Range, typically calculated over a 14-period interval.
- **Signals**:
  - **High Volatility**: ATR above a certain threshold, indicating significant price movements.
  - **Low Volatility**: ATR below a certain threshold, indicating minimal price movements.

## Combining Stochastic Oscillator and ATR
Using these indicators together can help confirm signals and reduce false positives:
- **Enhanced Accuracy**: While the Stochastic Oscillator helps identify momentum and potential reversals, ATR provides insight into market volatility, filtering out weak signals in low volatility environments.
- **Filter for False Signals**: The Stochastic Oscillator can sometimes give premature signals in volatile markets. By requiring confirmation through ATR's volatility measure, we filter out some of these false signals, increasing the likelihood that our signals are accurate.

## Methodology

- **Data Collection:** We collect historical price data for a selection of stocks over a specified period.
- **Indicator Calculation:** We calculate the Stochastic Oscillator and ATR for each stock.
- **Stock Screening:** We screen stocks based on the following criteria:
  - **Buy Criteria:** Stochastic Oscillator indicates an oversold condition and a cross of %K above %D, with ATR indicating a high volatility environment.
  - **Sell Criteria:** Stochastic Oscillator indicates an overbought condition and a cross of %K below %D, with ATR indicating a high volatility environment.
- **Visualization and Export:** We visualize the results and export the screened stocks to a CSV file for further analysis.

## Data Source and Flexibility

In this analysis, we will be using data from the FTSE 250 companies. However, feel free to use any dataset you prefer by simply changing the CSV file name in the script. Ensure that your dataset contains a column named "Ticker".

## Conclusion

This project aims to leverage the complementary strengths of the Stochastic Oscillator and ATR to create an intriguing stock screening tool. By considering both momentum and market volatility, our signals offer interesting insights designed to inspire further analysis. The chosen thresholds and periods are standard and widely accepted in technical analysis, providing a thought-provoking starting point for potential future exploration.



In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import contextlib
import sys
import logging 
import contextlib
import sys
import os
from tqdm import tqdm

# Function to suppress print statements
@contextlib.contextmanager
def suppress_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


# Function to calculate Average True Range (ATR)
def calculate_atr(data, window=14):
    data['High-Low'] = data['High'] - data['Low']
    data['High-Close'] = np.abs(data['High'] - data['Close'].shift(1))
    data['Low-Close'] = np.abs(data['Low'] - data['Close'].shift(1))
    data['True Range'] = data[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    data['ATR'] = data['True Range'].rolling(window=window).mean()
    return data

# Function to calculate Stochastic Oscillator
def calculate_stochastic_oscillator(data, window=14):
    data['Low_Min'] = data['Low'].rolling(window=window).min()
    data['High_Max'] = data['High'].rolling(window=window).max()
    data['Stochastic %K'] = 100 * ((data['Close'] - data['Low_Min']) / (data['High_Max'] - data['Low_Min']))
    data['Stochastic %D'] = data['Stochastic %K'].rolling(window=3).mean()
    return data

# Define threshold parameters
atr_threshold = 1.2  # Multiplier for mean ATR to determine high volatility
stochastic_overbought = 70  # Threshold for Stochastic %K and %D to determine overbought condition
stochastic_oversold = 30  # Threshold for Stochastic %K and %D to determine oversold condition

# Function to analyze stock for ATR and Stochastic Oscillator
def analyze_stock(symbol):
    try:
        # Fetch historical price data for the last 7 months
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=9*30)  # Approx. 7 months

        # Suppress yfinance output
        with suppress_output():
            data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        

                
        #data = yf.download(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
        
        if data.empty:
            print(f"No data fetched for {symbol}")
            return None

        # Calculate ATR and Stochastic Oscillator
        data = calculate_atr(data)
        data = calculate_stochastic_oscillator(data)

        data.dropna(subset=['ATR', 'Stochastic %K', 'Stochastic %D'], inplace=True)

        if data.empty:
            print(f"Insufficient data after calculating indicators for {symbol}")
            return None

        # Calculate whether the stock meets the high volatility criteria
        latest_atr = data['ATR'].iloc[-1]
        mean_atr = data['ATR'].mean()
        is_high_volatility = latest_atr > atr_threshold * mean_atr

        # Calculate whether the stock meets the overbought/oversold criteria
        latest_stochastic_k = data['Stochastic %K'].iloc[-1]
        latest_stochastic_d = data['Stochastic %D'].iloc[-1]
        is_overbought = latest_stochastic_k > stochastic_overbought and latest_stochastic_d > stochastic_overbought
        is_oversold = latest_stochastic_k < stochastic_oversold and latest_stochastic_d < stochastic_oversold

        # Determine if the stock meets at least two of the criteria
        criteria_met = sum([is_high_volatility, is_overbought, is_oversold])
        meets_criteria = criteria_met >= 2

        return {
            "Symbol": symbol,
            "Data": data,
            "ATR": latest_atr,
            "Stochastic %K": latest_stochastic_k,
            "Stochastic %D": latest_stochastic_d,
            "High Volatility": is_high_volatility,
            "Overbought": is_overbought,
            "Oversold": is_oversold,
            "Meets Criteria": meets_criteria  # Only true if at least two criteria are met
        }

    except Exception as e:
        print(f"Error analyzing stock {symbol}: {e}")
        return None




# Load tickers from the CSV file
tickers_df = pd.read_csv('ftse_250_tickers.csv')
symbols = tickers_df['Ticker'].tolist()

# Initialize list to store results
results = []

# Analyze each stock
for symbol in tqdm(symbols, desc="Processing stocks"):
    stock_data = analyze_stock(symbol)
    if stock_data is not None and stock_data["Meets Criteria"]:  # Only include stocks meeting the criteria
        results.append(stock_data)

# Convert results to DataFrame
results_df = pd.DataFrame([{
    "Symbol": r["Symbol"],
    "ATR": r["ATR"],
    "Stochastic %K": r["Stochastic %K"],
    "Stochastic %D": r["Stochastic %D"],
    "High Volatility": r["High Volatility"],
    "Overbought": r["Overbought"],
    "Oversold": r["Oversold"],
    "Meets Criteria": r["Meets Criteria"]
} for r in results])

# Export results to CSV
results_df.to_csv("atr_stochastic_results.csv", index=False)

# Print results
results_df




In [ ]:
# Calculate and print the detailed results
total_tickers = len(tickers_df)
tickers_meeting_criteria = len(results_df)
percentage_meeting_criteria = (tickers_meeting_criteria / total_tickers) * 100

# Print results
print("My preference is a 5-20% threshold which strikes a balance, ensuring a\nfocused yet diverse selection of stocks to allow further investigations.")
print(f"Tickers meeting criteria: {tickers_meeting_criteria}")
print(f"Percentage of shares that pass the criteria: {percentage_meeting_criteria:.2f}%")

# Determine if the percentage is within the desired filter range
if 5 <= percentage_meeting_criteria <= 20:
    print("The percentage of shares meeting the criteria is within the desirable range (5-20%).")
else:
    print("The percentage of shares meeting the criteria is outside the desirable range (5-15%).")

In [ ]:
# Function to print basic financial data
def print_financial_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    def format_value(value, currency=False):
        if isinstance(value, (int, float)):
            formatted_value = f"{value:,}"
            if currency:
                formatted_value = f"£{formatted_value}"
            return formatted_value
        return value

    market_cap = format_value(info.get('marketCap', 'N/A'), currency=True)
    shares_outstanding = format_value(info.get('sharesOutstanding', 'N/A'))
    year_high = info.get('fiftyTwoWeekHigh', 'N/A')
    year_low = info.get('fiftyTwoWeekLow', 'N/A')
    pe_ratio = info.get('trailingPE', 'N/A')
    eps = info.get('trailingEps', 'N/A')
    dividend_yield = info.get('dividendYield', 'N/A')
    payout_ratio = info.get('payoutRatio', 'N/A')
    revenue = format_value(info.get('totalRevenue', 'N/A'), currency=True)
    gross_profit = format_value(info.get('grossProfits', 'N/A'), currency=True)
    net_income = format_value(info.get('netIncomeToCommon', 'N/A'), currency=True)
    total_debt = format_value(info.get('totalDebt', 'N/A'), currency=True)
    operating_cash_flow = format_value(info.get('operatingCashflow', 'N/A'), currency=True)
    free_cash_flow = format_value(info.get('freeCashflow', 'N/A'), currency=True)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    description = info.get('longBusinessSummary', 'N/A')

    print(f"{'Ticker:':<25} {ticker}")
    print(f"{'Company:':<25} {info.get('shortName', 'N/A')}")
    print(f"{'Sector:':<25} {sector}")
    print(f"{'Industry:':<25} {industry}")
    print(f"{'Market Cap:':<25} {market_cap}")
    print(f"{'Shares Outstanding:':<25} {shares_outstanding}")
    print(f"{'52-Week High:':<25} {year_high}")
    print(f"{'52-Week Low:':<25} {year_low}")
    print(f"{'PE Ratio (TTM):':<25} {pe_ratio}")
    print(f"{'EPS (TTM):':<25} {eps}")
    print(f"{'Dividend Yield:':<25} {dividend_yield}")
    print(f"{'Payout Ratio:':<25} {payout_ratio}")
    print(f"{'Revenue (TTM):':<25} {revenue}")
    print(f"{'Gross Profit (TTM):':<25} {gross_profit}")
    print(f"{'Net Income (TTM):':<25} {net_income}")
    print(f"{'Total Debt:':<25} {total_debt}")
    print(f"{'Operating Cash Flow (TTM):':<25} {operating_cash_flow}")
    print(f"{'Free Cash Flow (TTM):':<25} {free_cash_flow}")
    print(f"\n{'Description:':<25} {description}")  # Print the full description
    print("-" * 50)


def plot_combined_indicators(data, symbol, atr_threshold, stochastic_overbought, stochastic_oversold):
    fig, ax1 = plt.subplots(figsize=(14, 6))

    # Plot Close Price
    ax1.plot(data.index, data['Close'], label='Close Price', color='blue')
    ax1.set_ylabel('Close Price', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_title(f'{symbol} - Close Price and ATR')
    ax1.grid(True)

    # Create a secondary y-axis for ATR
    ax2 = ax1.twinx()
    ax2.plot(data.index, data['ATR'], label='ATR', color='orange')
    atr_threshold_value = atr_threshold * data['ATR'].mean()
    ax2.axhline(y=atr_threshold_value, color='red', linestyle='--', linewidth=2, label=f'ATR Threshold ({atr_threshold_value:.2f})')
    ax2.set_ylabel('ATR', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')

    # Add legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

    # Plot Stochastic Oscillator on a separate subplot
    fig, ax3 = plt.subplots(figsize=(14, 4))
    ax3.plot(data.index, data['Stochastic %K'], label='Stochastic %K', color='green')
    ax3.plot(data.index, data['Stochastic %D'], label='Stochastic %D', color='red')
    ax3.axhline(y=stochastic_overbought, color='magenta', linestyle='--', linewidth=2, label='Overbought Threshold (70)')
    ax3.axhline(y=stochastic_oversold, color='cyan', linestyle='--', linewidth=2, label='Oversold Threshold (30)')
    ax3.set_title(f'{symbol} - Stochastic Oscillator')
    ax3.set_ylabel('Stochastic Value')
    ax3.grid(True)
    ax3.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

# Print the introductory text
print("The following stocks have met the specified criteria. A plot is displayed showing the stock price and its relationship to the associated indicators. Additionally, detailed financial and company information is provided for each stock.\n")

# Iterate through all tickers to plot graphs and print financial data
for index, row in results_df.iterrows():
    symbol = row["Symbol"]
    stock_data = next(item for item in results if item["Symbol"] == symbol)["Data"]
    plot_combined_indicators(stock_data, symbol, atr_threshold, stochastic_overbought, stochastic_oversold)
    print_financial_data(symbol)
